In [ ]:
# Notebook cell: génère un Page Bundle Hugo par commune
# pip install pandas plotly

import pandas as pd
import re
import os
from pathlib import Path
import html

# --- Configuration ---
CSV_PATH = "data/Eure_et_Loire.csv"   # chemin vers ton CSV
OUTPUT_BASE = Path("content/communes")  # racine des bundles Hugo
OUTPUT_BASE.mkdir(parents=True, exist_ok=True)
ENCODING = "utf-8"

# Colonnes à considérer comme "hiérarchie"
HIER_COLS_PREFERRED = [
    "insee","section","superficie","intendance","election","subdelegation",
    "grenier","coutume","parlement","bailliage","gouvernement",
    "diocese","archidiacone","doyenne","vocable","presentateur",
    "district_1790","canton_1790","canton_1801","arrondissement_1982","canton _1982"
]

# --- fonctions utilitaires ---

def detect_year_from_col(colname):
    """Retourne l'année trouvée dans le nom de colonne (ex: 'V_1790_f' -> 1790)"""
    m = re.search(r'(1[0-9]{3}|17[0-9]{2}|18[0-9]{2}|19[0-9]{2}|20[0-9]{2}|\d{3,4})', colname)
    if m:
        y = int(m.group(0))
        if 800 <= y <= 2100:
            return y
    return None

def sanitize_numeric(val):
    """Nettoie 'n_c', 'lac.', '29!' etc. et renvoie un int ou None"""
    if pd.isna(val):
        return None
    s = str(val).strip()
    if s in {"n_c", "lac.", "lac", "", "-", "?"}:
        return None
    m = re.search(r'(-?\d+)', s.replace(" ", ""))
    if m:
        try:
            return int(m.group(1))
        except:
            return None
    return None

def safe_dirname(s):
    """Nom de dossier Hugo sûr (sans espaces, accents, ni ponctuation problématique)"""
    s = str(s).strip().lower().replace(" ", "_")
    s = re.sub(r'[^a-z0-9_\-]', '', s)
    return s or "commune"

# --- Lire le CSV ---
df = pd.read_csv(CSV_PATH, encoding=ENCODING, dtype=str).head(2)
df.columns = [c.strip() for c in df.columns]

# Colonnes population
pop_cols = {c: detect_year_from_col(c) for c in df.columns}
pop_cols = {c: y for c,y in pop_cols.items() if y is not None}
pop_cols = dict(sorted(pop_cols.items(), key=lambda kv: kv[1]))

# Colonnes hiérarchiques
hier_cols = [c for c in HIER_COLS_PREFERRED if c in df.columns]
if not hier_cols:
    exclude = set(pop_cols.keys()) | {"nom", "insee"}
    hier_cols = [c for c in df.columns if c not in exclude][:12]

# --- Génération des bundles ---
for idx, row in df.iterrows():
    commune_nom = row.get("nom") or f"commune_{idx}"
    insee = row.get("insee") or ""
    title = str(commune_nom)

    # Créer dossier ../content/communes/<NomCommune>
    dirname = safe_dirname(commune_nom)
    commune_dir = OUTPUT_BASE / dirname
    commune_dir.mkdir(parents=True, exist_ok=True)
    qmd_path = commune_dir / "index.qmd"

    # Table population
    data_pairs = []
    for col, year in pop_cols.items():
        v = sanitize_numeric(row.get(col, ""))
        data_pairs.append((int(year), (int(v) if v is not None else None)))

    # Table hiérarchique
    hier_table_lines = ["| Propriété | Valeur |", "|---|---|"]
    for col in hier_cols:
        val = row.get(col, "")
        safe_val = "" if pd.isna(val) else html.escape(str(val))
        hier_table_lines.append(f"| {col} | {safe_val} |")

    # YAML Hugo/Quarto
    yaml_header = [
        "---",
        f'title: "{title}"',
        "format: html",
        "execute:",
        "  evaluate: true",
        "---",
        ""
    ]

    # Corps QMD
    body_lines = []
    body_lines.append(f"# {title}\n")
    body_lines.append("## Informations hiérarchiques\n")
    body_lines.extend(hier_table_lines)
    body_lines.append("\n## Évolution historique de la population\n")
    body_lines.append("\n```{python}\nimport pandas as pd\nimport plotly.express as px\n\npairs = " + repr(data_pairs) + "\ndf_pop = pd.DataFrame(pairs, columns=['year','population']).sort_values('year')\ndf_pop['population'] = pd.to_numeric(df_pop['population'], errors='coerce')\ndf_plot = df_pop.dropna(subset=['population'])\n\nif df_plot.empty:\n    print('Aucune donnée disponible.')\nelse:\n    fig = px.line(df_plot, x='year', y='population', markers=True,\n                  title='Population par année')\n    fig.update_layout(xaxis_title='Année', yaxis_title='Population')\n    fig.show()\n```\n")

    with open(qmd_path, "w", encoding=ENCODING) as f:
        f.write("\n".join(yaml_header + body_lines))

    print(f"Créé: {qmd_path}")

print("✅ Tous les bundles Hugo sont générés dans:", OUTPUT_BASE.resolve())
